# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
import numpy as np
import sklearn.model_selection as ms
import harness
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA

In [8]:
pd.options.display.max_columns=None

In [3]:
def train_test_split():
    flights = pd.read_csv('data/flights_1%.csv')
    flights_train, flights_test = ms.train_test_split(
        flights, test_size=0.2, random_state=729
    )
    flights_train.to_csv('data/flights_train.csv', index_label='id')
    flights_test.to_csv('data/flights_test.csv', index_label='id')

In [4]:
#train_test_split()

In [5]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [6]:
weather_0 = pd.read_csv('weather_0.csv', index_col=0)
weather_1 = pd.read_csv('weather_1.csv', index_col=0)
weather_2 = pd.read_csv('weather_2.csv', index_col=0)
weather_3 = pd.read_csv('weather_3.csv', index_col=0)
weather_4 = pd.read_csv('weather_4.csv', index_col=0)
weather = pd.concat([weather_0,weather_1,weather_2,weather_3,weather_4])

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [7]:
flights_clean = harness.clean_train(flights_train)

In [8]:
flights_clean_copy = flights_clean.copy()
flights_clean_copy['month']=flights_clean_copy.fl_date
flights_clean_copy.month = flights_clean_copy.month.map(lambda v: int(v[5:7]))
flights_clean_copy['day']=flights_clean_copy.fl_date
flights_clean_copy.day = flights_clean_copy.day.map(lambda v: int(v[8:]))
flights_clean_copy['haul']=flights_clean_copy.crs_elapsed_time/60
flights_clean_copy['haul'] = pd.cut(flights_clean_copy.haul,bins=[0,3,6,12],labels=['Short','Medium','Long'])

In [9]:
flights_with_weather = flights_clean_copy.merge(
    weather, left_on=['fl_date', 'origin_city_name'], right_on=['date', 'city']
).merge(
    weather, left_on=['fl_date', 'dest_city_name'], right_on=['date', 'city'],
    suffixes=('_origin', '_dest')
)

weather_category_map = {
    'Partially cloudy': 'Cloudy',
    'Clear': 'Sunny',
    'Rain, Partially cloudy': 'Rainy',
    'Rain, Overcast': 'Rainy',
    'Overcast': 'Cloudy',
    'Rain': 'Rainy',
    'Snow, Partially cloudy': 'Snowy',
    'Snow, Overcast': 'Snowy',
    'Snow': 'Snowy',
}
flights_with_weather['weather_origin'] = flights_with_weather.conditions_origin.map(weather_category_map)
flights_with_weather['weather_dest'] = flights_with_weather.conditions_dest.map(weather_category_map)

In [10]:
flights_with_weather = flights_clean_copy.merge(flights_with_weather, how='outer')

In [11]:
flights_with_weather['conditions_origin'] = flights_with_weather['conditions_origin'].fillna('Unknown')
flights_with_weather['conditions_dest'] = flights_with_weather['conditions_dest'].fillna('Unknown')
flights_with_weather['weather_origin'] = flights_with_weather['weather_origin'].fillna('Unknown')
flights_with_weather['weather_dest'] = flights_with_weather['weather_dest'].fillna('Unknown')

In [12]:
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'conditions_origin')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'conditions_dest')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'weather_origin')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'weather_dest')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'op_unique_carrier')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'fl_date')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'dest_city_name')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'origin_airport_id')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'origin_city_name')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'dest_airport_id')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'day')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'month')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'haul')
flights_with_weather = harness.add_grouped_stats(flights_with_weather,'tail_num')

In [ ]:
flights_with_weather = flights_with_weather.drop(['conditions_origin',
                                                  'conditions_dest',
                                                  'weather_origin', 'date_dest',
                                                  'weather_dest', 'dup',
                                                  'op_unique_carrier', 'date_origin',
                                                  'fl_date','dest_city_name',
                                                  'origin_airport_id','origin_city_name',
                                                  'dest_airport_id','day','month','haul',
                                                  'branded_code_share',
                                                  'mkt_carrier','origin','dest',
                                                  'tail_num','mkt_unique_carrier'], axis =1)

In [ ]:
flights_with_weather['tail_num_delay_std'] = flights_with_weather['tail_num_delay_std'].fillna(0)
flights_with_weather['dest_airport_id_delay_std'] = flights_with_weather['dest_airport_id_delay_std'].fillna(0)
flights_with_weather['origin_city_name_delay_std'] = flights_with_weather['origin_city_name_delay_std'].fillna(0)
flights_with_weather['origin_airport_id_delay_std'] = flights_with_weather['origin_airport_id_delay_std'].fillna(0)
flights_with_weather['dest_city_name_delay_std'] = flights_with_weather['dest_city_name_delay_std'].fillna(0)


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [ ]:
pca = PCA()
pca.fit(flights_with_weather)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)

In [ ]:
n_components = 6

In [ ]:
pca = PCA(n_components=n_components)
components = pca.fit_transform(flights_with_weather)

total_var = pca.explained_variance_ratio_.sum() * 100

labels = {str(i): f"PC {i+1}" for i in range(n_components)}
labels['color'] = 'Median Price'

fig = px.scatter_matrix(
    components,
    dimensions=range(n_components),
    labels=labels,
    title=f'Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline
print(__doc__)

# Code source: Tyler Lanigan <tylerlanigan@gmail.com>
#              Sebastian Raschka <mail@sebastianraschka.com>

# License: BSD 3 clause

RANDOM_STATE = 42
FIG_SIZE = (10, 7)


features, target = load_wine(return_X_y=True)

# Make a train/test split using 30% test size
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=0.30,
                                                    random_state=RANDOM_STATE)

# Fit to data and predict using pipelined GNB and PCA.
unscaled_clf = make_pipeline(PCA(n_components=2), GaussianNB())
unscaled_clf.fit(X_train, y_train)
pred_test = unscaled_clf.predict(X_test)

# Fit to data and predict using pipelined scaling, GNB and PCA.
std_clf = make_pipeline(StandardScaler(), PCA(n_components=2), GaussianNB())
std_clf.fit(X_train, y_train)
pred_test_std = std_clf.predict(X_test)

# Show prediction accuracies in scaled and unscaled data.
print('\nPrediction accuracy for the normal test dataset with PCA')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test)))

print('\nPrediction accuracy for the standardized test dataset with PCA')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test_std)))

# Extract PCA from pipeline
pca = unscaled_clf.named_steps['pca']
pca_std = std_clf.named_steps['pca']

# Show first principal components
print('\nPC 1 without scaling:\n', pca.components_[0])
print('\nPC 1 with scaling:\n', pca_std.components_[0])

# Use PCA without and with scale on X_train data for visualization.
X_train_transformed = pca.transform(X_train)
scaler = std_clf.named_steps['standardscaler']
X_train_std_transformed = pca_std.transform(scaler.transform(X_train))

# visualize standardized vs. untouched dataset with PCA performed
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=FIG_SIZE)


for l, c, m in zip(range(0, 3), ('blue', 'red', 'green'), ('^', 's', 'o')):
    ax1.scatter(X_train_transformed[y_train == l, 0],
                X_train_transformed[y_train == l, 1],
                color=c,
                label='class %s' % l,
                alpha=0.5,
                marker=m
                )

for l, c, m in zip(range(0, 3), ('blue', 'red', 'green'), ('^', 's', 'o')):
    ax2.scatter(X_train_std_transformed[y_train == l, 0],
                X_train_std_transformed[y_train == l, 1],
                color=c,
                label='class %s' % l,
                alpha=0.5,
                marker=m
                )

ax1.set_title('Training dataset after PCA')
ax2.set_title('Standardized training dataset after PCA')

for ax in (ax1, ax2):
    ax.set_xlabel('1st principal component')
    ax.set_ylabel('2nd principal component')
    ax.legend(loc='upper right')
    ax.grid()

plt.tight_layout()

plt.show()

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

Below selective linear is copied to test harness one hot encode functions.

In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns=None

import harness

np.warnings.filterwarnings('ignore', 'Ill-conditioned matrix')

In [2]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [3]:
flights_clean = harness.clean_train(flights_train)


In [ ]:
flights_clean = harness.clean_train(flights_train)

add_features = harness.chain(
    harness.add_date_parts, harness.add_haul, harness.add_weather
)

flights_with_features = add_features(flights_clean)

x_transform = harness.chain(
    harness.keep_only_test_columns,
    add_features,
    [harness.add_all_grouped_stats, flights_with_features],
    harness.only_numeric,
    harness.scale,
)

transformer = harness.DataTransformer(
    x_transform
)

In [ ]:
for col in x_train.columns:
    print(f"{col}: {np.corrcoef(x_train[col], y_train.arr_delay)[0, 1]}")

In [ ]:
def select_columns(df):
    return df[[
        'origin_airport_id_delay_mean',
        'origin_airport_id_delay_std',
        'dest_airport_id_delay_mean',
        'dest_airport_id_delay_std',
    ]]

def train_model():
    linreg = lm.Ridge(random_state=42)
    grid_search = ms.GridSearchCV(
        linreg, dict(alpha=[10 ** i for i in range(-6, 7)]),
    )
    grid_search.fit(x_train.values, y_train.values)
    print(grid_search.best_score_)
    harness.save(grid_search.best_estimator_, 'selective_linear_model')

In [ ]:
x_transform = harness.chain(x_transform, select_columns)
transformer = harness.DataTransformer(
    x_transform
)
x_train, y_train = transformer.extract_transform(flights_clean)

In [ ]:
#train_model()

In [ ]:
linreg = harness.load('selective_linear_model')
linreg_model = harness.TrainedModel(linreg, transformer)

In [ ]:
linreg_model.validate(harness.clean_train(flights_test))

In [ ]:
# linreg_model.submit(
#     'data/test.csv', 'everything_linear_submission.csv', 'predicted_delay'
# )

In [5]:
def make_city_dummies(df):
    cols = [
        'origin_city_name', 'dest_city_name',
        'origin_airport_id', 'dest_airport_id',
    ]
    for col in cols:
        dummy = pd.get_dummies(df[col],prefix=col)
        df = pd.concat([df,dummy], axis=1)
    return df

In [4]:
flights_copy = flights_clean.copy()

flights_copy['origin_dest_city_name'] = flights_copy['origin_city_name']+' to '+flights_copy['dest_city_name']

flights_copy


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,origin_dest_city_name
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
121941,2019-08-25,AA,AA_CODESHARE,AA,5636,OH,N723PS,5636,11057,CLT,"Charlotte, NC",10208,AGS,"Augusta, GA",1818,1811.0,-7.0,14.0,1825.0,1856.0,3.0,1921,1859.0,-22.0,0,0,N,63,48.0,31.0,1,140,"Charlotte, NC to Augusta, GA"
109932,2018-03-02,WN,WN,WN,2028,WN,N244WN,2028,13495,MSY,"New Orleans, LA",10693,BNA,"Nashville, TN",745,738.0,-7.0,7.0,745.0,851.0,4.0,915,855.0,-20.0,0,0,N,90,77.0,66.0,1,471,"New Orleans, LA to Nashville, TN"
63108,2019-03-06,UA,UA,UA,545,UA,N69804,545,11292,DEN,"Denver, CO",12266,IAH,"Houston, TX",1214,1233.0,19.0,12.0,1245.0,1531.0,4.0,1531,1535.0,4.0,0,0,N,137,122.0,106.0,1,862,"Denver, CO to Houston, TX"
91519,2019-10-11,DL,DL_CODESHARE,DL,3798,OO,N452SW,3798,16869,XWA,"Williston, ND",13487,MSP,"Minneapolis, MN",600,609.0,9.0,31.0,640.0,817.0,3.0,806,820.0,14.0,0,0,N,126,131.0,97.0,1,553,"Williston, ND to Minneapolis, MN"
8361,2019-09-26,UA,UA_CODESHARE,UA,5359,OO,N120SY,5359,10713,BOI,"Boise, ID",14771,SFO,"San Francisco, CA",1426,1431.0,5.0,10.0,1441.0,1504.0,7.0,1521,1511.0,-10.0,0,0,N,115,100.0,83.0,1,522,"Boise, ID to San Francisco, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48069,2019-05-01,G4,G4,G4,2170,G4,260NV,2170,14761,SFB,"Sanford, FL",11973,GPT,"Gulfport/Biloxi, MS",605,559.0,-6.0,13.0,612.0,627.0,5.0,638,632.0,-6.0,0,0,N,93,93.0,75.0,1,485,"Sanford, FL to Gulfport/Biloxi, MS"
68804,2019-07-01,UA,UA_CODESHARE,UA,6259,YV,N80343,6259,10140,ABQ,"Albuquerque, NM",12266,IAH,"Houston, TX",540,553.0,13.0,12.0,605.0,853.0,3.0,851,856.0,5.0,0,0,N,131,123.0,108.0,1,744,"Albuquerque, NM to Houston, TX"
43542,2018-07-12,AA,AA,AA,412,AA,N177US,412,11057,CLT,"Charlotte, NC",14107,PHX,"Phoenix, AZ",1300,1313.0,13.0,13.0,1326.0,1408.0,5.0,1415,1413.0,-2.0,0,0,N,255,240.0,222.0,1,1773,"Charlotte, NC to Phoenix, AZ"


In [10]:
#df['origin_dest_airport_id'] = df['origin_airport_id']+' to '+df['dest_airport_id']

    
flights_copy = flights_clean.copy()

flights_copy['origin_dest_airport_id'] = flights_copy['origin_airport_id'].map(lambda v: str(v))+' to '+flights_copy['dest_airport_id'].map(lambda v: str(v))
    
df['origin_dest_airport_id']  =df['origin_airport_id'].map(lambda v: str(v))+' to '+df['dest_airport_id'].map(lambda v: str(v))
    
df['origin_dest_airport_id'] = df['origin_airport_id']+' to '+df['dest_airport_id']

flights_copy


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,origin_dest_airport_id
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
121941,2019-08-25,AA,AA_CODESHARE,AA,5636,OH,N723PS,5636,11057,CLT,"Charlotte, NC",10208,AGS,"Augusta, GA",1818,1811.0,-7.0,14.0,1825.0,1856.0,3.0,1921,1859.0,-22.0,0,0,N,63,48.0,31.0,1,140,11057 to 10208
109932,2018-03-02,WN,WN,WN,2028,WN,N244WN,2028,13495,MSY,"New Orleans, LA",10693,BNA,"Nashville, TN",745,738.0,-7.0,7.0,745.0,851.0,4.0,915,855.0,-20.0,0,0,N,90,77.0,66.0,1,471,13495 to 10693
63108,2019-03-06,UA,UA,UA,545,UA,N69804,545,11292,DEN,"Denver, CO",12266,IAH,"Houston, TX",1214,1233.0,19.0,12.0,1245.0,1531.0,4.0,1531,1535.0,4.0,0,0,N,137,122.0,106.0,1,862,11292 to 12266
91519,2019-10-11,DL,DL_CODESHARE,DL,3798,OO,N452SW,3798,16869,XWA,"Williston, ND",13487,MSP,"Minneapolis, MN",600,609.0,9.0,31.0,640.0,817.0,3.0,806,820.0,14.0,0,0,N,126,131.0,97.0,1,553,16869 to 13487
8361,2019-09-26,UA,UA_CODESHARE,UA,5359,OO,N120SY,5359,10713,BOI,"Boise, ID",14771,SFO,"San Francisco, CA",1426,1431.0,5.0,10.0,1441.0,1504.0,7.0,1521,1511.0,-10.0,0,0,N,115,100.0,83.0,1,522,10713 to 14771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48069,2019-05-01,G4,G4,G4,2170,G4,260NV,2170,14761,SFB,"Sanford, FL",11973,GPT,"Gulfport/Biloxi, MS",605,559.0,-6.0,13.0,612.0,627.0,5.0,638,632.0,-6.0,0,0,N,93,93.0,75.0,1,485,14761 to 11973
68804,2019-07-01,UA,UA_CODESHARE,UA,6259,YV,N80343,6259,10140,ABQ,"Albuquerque, NM",12266,IAH,"Houston, TX",540,553.0,13.0,12.0,605.0,853.0,3.0,851,856.0,5.0,0,0,N,131,123.0,108.0,1,744,10140 to 12266
43542,2018-07-12,AA,AA,AA,412,AA,N177US,412,11057,CLT,"Charlotte, NC",14107,PHX,"Phoenix, AZ",1300,1313.0,13.0,13.0,1326.0,1408.0,5.0,1415,1413.0,-2.0,0,0,N,255,240.0,222.0,1,1773,11057 to 14107


### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.